In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
import copy
from matplotlib import pylab as plt
import cv2
from os import path
%matplotlib inline

In [2]:
def getArrayFromPoint(points,text_name):
    return points[points.text_digits==text_name][['x','y','radius','text_digits']].iloc[0].values
def getArrayFromPoints(points):
    return points[['x','y','radius','text_digits']].values

In [3]:
def route_dist(route):
    dist = start_distances[route[0]]
    for p1,p2 in zip(route[:-1],route[1:]):
        dist += distance_matrix[p1,p2]
    return dist + finish_distances[route[-1]]

def distance(x,y):
    return np.sqrt((float(x[0])-float(y[0]))**2  + (float(x[1])-float(y[1]))**2)

def register_top(route,dist):
    global top
    global routes_to_find
    global porog
    top.append((points[route],dist))
    top = sorted(top,key = lambda x: x[1])[:routes_to_find]
    #print 'new top'
    try:
        last = top[routes_to_find-1]
        #print 'porog changed'
        porog = last[1]
    except:
        pass
    
def best_route(route,dist,verbose=True):
    current = route[-1]
    if dist+porog_adjust[len(points)-len(route)] > porog:
        return
    if dist+finish_distances[current]>porog:
        return
    if len(route) == len(points):
        register_top(route,dist+finish_distances[current])
        return
    points_walked = 0
    if (len(route)==4) and (verbose==True):
        print (points[route][:,3], top[0][1] if len(top)> 0 else 2000000000 )
    for i in sorted_by_distance_points[current]:
        if i in route:
            continue
        new_route = copy.copy(route)
        new_route.append(i)
        new_dist = dist+distance_matrix[current,i]
        if new_dist>porog:
            continue
        best = best_route(new_route,new_dist,verbose=verbose)
        points_walked+=1
        if points_walked == nearest_points:
            break

In [33]:
start_name = '6'
finish_name = 'SK'
kp_in = '7-22'
dat_filename = 'maps/2019o/1v.png.dat'
map_filename = 'maps/2019o/1v.png'
mashtab = 33333
nearest_points = 8
routes_to_find = 10
penalty_allowed = 0
kps_to_skip = 0
outfolder = 'output/'

In [34]:
img = cv2.imread(map_filename)
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
st_297 = max(img.shape[0],img.shape[1])
#factor = 1./((st_297/297)*mashtab/1000)
sms_per_pixel = 29.7/st_297
kms_per_pixel = mashtab*sms_per_pixel/100000
factor = kms_per_pixel

In [6]:
def kp_gen(kp_str):
    kp_strs = kp_str.split(',')
    for s in kp_strs:
        if '-' in s:
            v = s.split('-')
            min_s = int(v[0])
            max_s = int(v[1])
            for i in range(min_s,max_s+1,1):
                yield str(i)
        else:
            yield s

In [7]:
#инициализация. Нужно также указать начало и конец выбора

points = pd.read_csv(dat_filename,header=0)
start = getArrayFromPoint(points,start_name)
finish = getArrayFromPoint(points,finish_name)
kps = list(kp_gen(kp_in))
points = points[points.text_digits.isin(kps)]
points = getArrayFromPoints(points)
all_points = np.concatenate(([start],points,[finish]))


In [8]:
#prepare data
start_distances = np.array([distance(start,x) for x in points])
points = points[start_distances.argsort()]
start_distances = np.array([distance(start,x) for x in points])
distance_matrix = np.array([[distance(x,y) for x in points] for y in points])
finish_distances = np.array([distance(finish,x) for x in points])
sorted_by_distance_points = [distance_matrix[i].argsort() for i in range(len(points))]

nearest_peregons = distance_matrix.flatten()
nearest_peregons = nearest_peregons[nearest_peregons!=0]
nearest_peregons.sort()
nearest_peregons = nearest_peregons[::2]
nearest_peregons = nearest_peregons.cumsum()

porog_adjust = np.hstack([np.array([finish_distances.min()]), nearest_peregons+finish_distances.min()])

In [35]:
#real work here
top = []
porog=1e20
if kps_to_skip == 0:
    for first in (range(len(points))):
        b = best_route([first],start_distances[first])
else:
    all_best = []
    for combination in (itertools.combinations(range(1,len(kps),1),len(kps)-kps_to_skip)):
        points_ind = np.concatenate((combination,))
        points = all_points[points_ind]
        distance_matrix = np.array([[distance(x,y) for x in points] for y in points])
        start_distances = np.array([distance(start,x) for x in points])
        finish_distances = np.array([distance(finish,x) for x in points])
        sorted_by_distance_points = [distance_matrix[i].argsort() for i in range(len(points))]
        for first in range(len(points)):
            b = best_route([first],start_distances[first],verbose=False)   

['8' '9' '7' '10'] 2000000000
['8' '9' '7' '11'] 6685.147371912947
['8' '9' '7' '14'] 6655.646457306754
['8' '9' '7' '15'] 6655.646457306754
['8' '9' '7' '13'] 6655.646457306754
['8' '9' '10' '11'] 6655.646457306754
['8' '9' '10' '14'] 6655.646457306754
['8' '9' '10' '15'] 6655.646457306754
['8' '9' '10' '7'] 6655.646457306754
['8' '9' '10' '13'] 6655.646457306754
['8' '9' '10' '18'] 6655.646457306754
['8' '9' '10' '17'] 6655.646457306754
['8' '9' '14' '15'] 6655.646457306754
['8' '9' '14' '10'] 6655.646457306754
['8' '9' '14' '18'] 6655.646457306754
['8' '9' '14' '13'] 6655.646457306754
['8' '9' '14' '11'] 6655.646457306754
['8' '9' '14' '17'] 6655.646457306754
['8' '9' '14' '7'] 6655.646457306754
['8' '9' '14' '12'] 6655.646457306754
['8' '9' '13' '12'] 6655.646457306754
['8' '9' '13' '17'] 6655.646457306754
['8' '9' '13' '14'] 6655.646457306754
['8' '9' '13' '16'] 6655.646457306754
['8' '9' '13' '10'] 6655.646457306754
['8' '9' '13' '18'] 6655.646457306754
['8' '9' '13' '20'] 6655.6

['7' '11' '10' '15'] 6655.646457306754
['7' '11' '10' '9'] 6655.646457306754
['7' '11' '10' '13'] 6655.646457306754
['7' '11' '10' '8'] 6655.646457306754
['7' '11' '10' '18'] 6655.646457306754
['7' '11' '10' '17'] 6655.646457306754
['7' '11' '15' '14'] 6655.646457306754
['7' '11' '15' '10'] 6655.646457306754
['7' '11' '15' '18'] 6655.646457306754
['7' '11' '15' '9'] 6655.646457306754
['7' '11' '15' '17'] 6655.646457306754
['7' '11' '14' '15'] 6655.646457306754
['7' '11' '14' '10'] 6655.646457306754
['7' '11' '14' '9'] 6655.646457306754
['7' '11' '14' '18'] 6655.646457306754
['7' '11' '14' '13'] 6655.646457306754
['7' '11' '14' '17'] 6655.646457306754
['7' '11' '9' '8'] 6655.646457306754
['7' '11' '9' '10'] 6655.646457306754
['7' '11' '9' '14'] 6655.646457306754
['7' '11' '9' '13'] 6655.646457306754
['7' '8' '9' '10'] 6655.646457306754
['7' '8' '9' '14'] 6655.646457306754
['7' '8' '9' '13'] 6655.646457306754
['7' '8' '9' '11'] 6655.646457306754
['7' '8' '13' '12'] 6655.646457306754
['7'

In [36]:
def print_top():
    best = top[0][1]
    for t in top:
        print (np.concatenate([[start],t[0].astype(int),[finish]])[:,3],\
            '%.3f км'%(t[1]*factor),\
            '(-%.3f,'%((t[1]-best)*factor),\
            '%.2f%%)' % ((t[1]-best)*100./best))
        
def print_top_with_skipped():
    best = top[0][1]
    for t in top:
        print (np.concatenate([[start],t[0].astype(int),[finish]])[:,3],\
            (list(set(all_points[:,3]) - set(t[0][:,3]) - set([start[3],finish[3]]))),\
            '%.3f км'%(t[1]*factor),\
            '(-%.3f,'%((t[1]-best)*factor),\
            '%.2f%%)' % ((t[1]-best)*100./best))
        
print_top()

['6' 8 9 7 11 10 14 15 18 17 13 12 16 19 20 21 22 'SK'] 19.506 км (-0.000, 0.00%)
['6' 8 9 7 11 10 14 15 18 21 22 20 17 13 12 16 19 'SK'] 19.519 км (-0.014, 0.07%)
['6' 8 9 7 10 11 15 14 18 17 13 12 16 19 20 21 22 'SK'] 19.592 км (-0.086, 0.44%)
['6' 8 9 7 10 11 15 14 18 21 22 20 17 13 12 16 19 'SK'] 19.606 км (-0.100, 0.51%)
['6' 8 9 7 11 10 15 14 18 17 13 12 16 19 20 21 22 'SK'] 19.619 км (-0.113, 0.58%)
['6' 8 9 7 11 10 15 14 18 21 22 20 17 13 12 16 19 'SK'] 19.632 км (-0.127, 0.65%)
['6' 8 9 7 10 11 15 14 13 12 16 17 18 21 22 20 19 'SK'] 19.838 км (-0.332, 1.70%)
['6' 8 9 7 11 10 15 14 13 12 16 17 18 21 22 20 19 'SK'] 19.865 км (-0.359, 1.84%)
['6' 8 9 7 10 11 14 15 18 17 13 12 16 19 20 21 22 'SK'] 19.879 км (-0.374, 1.92%)
['6' 8 9 7 10 11 14 15 18 21 22 20 17 13 12 16 19 'SK'] 19.893 км (-0.387, 1.99%)


### Нарисуем!

In [242]:
save_top()

In [230]:
def save_top():
    for i,t in enumerate(top):
        new_image = copy.copy(img)
        route = np.vstack([[start],t[0],[finish]])
        for p1,p2 in zip(route[:-1],route[1:]):
            lineThickness = 7
            new_image = cv2.line(new_image, (p1[0], p1[1]), (p2[0], p2[1]), (255,0,0), lineThickness)
        filename = '%d. %s (%.3f km).jpg' % (i+1,'-'.join(np.concatenate([[start],t[0],[finish]])[:,3]),t[1]*factor)
        new_image = cv2.cvtColor(new_image,cv2.COLOR_RGB2BGR)
        cv2.imwrite(path.join(outfolder,filename), new_image)